<h2> 분석 </h2>


* 머신러닝 학습시 date는 인식이 안됨, NAN 도 안됨

$\Rightarrow$ null 값을 처리해야 함

$\Rightarrow$ 전혀 머신러닝 분석이 진행되지 않음

<h2>null 값 제거하기</h2>


* null 값을 제거할 때 column(세로)로 제거하면 중요도가 높은 강수량을 삭제하게 되어 성능이 떨어짐
* null 값을 제거할 때 row(가로)로 제거하면 데이터 수가 너무 줄어들음

$\Rightarrow$ null 값은 제거가 아니라 조정을 해야함

$\Rightarrow$ 머신러닝 결과 null 값을 column으로 제거한 모델이 약 0.63 정도로 성능이 가장 높음


<h2>null 값 조정하기</h2>


* 0으로 변경하기
* 평균값으로 변경하기
* 중간값으로 변경하기

$\Rightarrow$ null 값을 조정한 결과 정확도가 향상되었으며 0으로 변경한 모델은 71% 정도로 성능이 가장 높았다.

<h2>중간결과</h2>

|옵션|성능|
|---|---|
|null인 column 제거| 0.636750  |
|null인 row 제거| 0.57887  |
|null을 0으로 변경|  0.7012665 |
|null을 평균을 변경| 0.65259  |
|null을 중간값으로 변경| 0.680691  |

In [1]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from copy import deepcopy

#  데이터 가져오기

In [2]:
train_df=pd.read_csv('./data/train.csv')
test_df=pd.read_csv('./data/test.csv')

In [3]:
train_df_backup = deepcopy(train_df)
test_df_backup = deepcopy(test_df)

In [4]:
train_df.columns

Index(['date', 'precipitation', 'temp_mean', 'temp_highest', 'temp_lowest',
       'PM10', 'PM2.5', 'humidity', 'sunshine_sum', 'sunshine_rate',
       'wind_mean', 'wind_max', 'rental'],
      dtype='object')

* date: 날짜
* precipitation: 강수량(mm)
* temp_mean: 평균 기온(℃)
* temp_highest: 최고 기온(℃)
* temp_lowest: 최저 기온(℃)
* PM10: 미세먼지(㎍/㎥)
* PM2.5: 초미세먼지(㎍/㎥)
* humidity: 습도(%rh)
* sunshine_sum: 일조합
* sunshine_rate: 일조율
* wind_mean: 평균 풍속(m/s)
* wind_max: 최대 풍속(m/s)
* rentals : 따릉이 대여량

In [5]:
train_df.isnull().sum()

date               0
precipitation    678
temp_mean          0
temp_highest       0
temp_lowest        0
PM10              67
PM2.5             68
humidity           0
sunshine_sum       5
sunshine_rate      0
wind_mean          0
wind_max           0
rental             0
dtype: int64

In [6]:
test_df.isnull().sum()

date               0
precipitation    205
temp_mean          0
temp_highest       0
temp_lowest        0
PM10               0
PM2.5              0
humidity           0
sunshine_sum       1
sunshine_rate      0
wind_mean          0
wind_max           0
dtype: int64

In [7]:
random_state_list=[random.randint(0, 100) for _ in range(10)]
random_state_list

[93, 20, 6, 54, 59, 65, 10, 86, 88, 77]

# null 제거

## 결측치 행 제거 후 학습진행

In [8]:
train_df = deepcopy(train_df_backup)
train_df.dropna(axis=0, inplace=True)

In [9]:
X,y = train_df.iloc[:,1:-1].to_numpy(), train_df.iloc[:,-1].to_numpy()

In [10]:
X.shape, y.shape

((386, 11), (386,))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=123456)

In [12]:
for i, state in enumerate(random_state_list): 
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(f'{i+1}번째')
    print(f'Score : {score}')
    print(f'Oob_Score : {rf.oob_score_}')

1번째
Score : 0.6367504903166681
Oob_Score : 0.6516788633015427
2번째
Score : 0.6336329576037865
Oob_Score : 0.6458057572676962
3번째
Score : 0.6430593432915341
Oob_Score : 0.6406486652269996
4번째
Score : 0.6413284724852445
Oob_Score : 0.6343480568232255
5번째
Score : 0.6344640091294422
Oob_Score : 0.64614137497163
6번째
Score : 0.6401442747948944
Oob_Score : 0.6457919952312761
7번째
Score : 0.6455456873657099
Oob_Score : 0.6370225781984254
8번째
Score : 0.6198409431896846
Oob_Score : 0.6419870590685898
9번째
Score : 0.6155096735203907
Oob_Score : 0.6464987334508933
10번째
Score : 0.6247154534044148
Oob_Score : 0.6540086571009925


In [13]:
print('\033[31m'+'Feature importance'+'\033[0m')
print(*sorted(zip(train_df.iloc[:,1:-1].columns ,rf.feature_importances_), key=lambda x : x[1], reverse=True), sep='\n')

Feature importance
('temp_highest', 0.32360768580116955)
('precipitation', 0.28586481432304534)
('humidity', 0.06782735191597596)
('temp_mean', 0.06589073464750762)
('wind_mean', 0.05389977750129226)
('sunshine_sum', 0.04746082164874536)
('temp_lowest', 0.04208108337927266)
('sunshine_rate', 0.03803043049372001)
('wind_max', 0.02729710645038015)
('PM10', 0.026389042358724465)
('PM2.5', 0.021651151480166723)


교차검증시 데이터부족으로 인한 학습부진 발생, 교차검증 점수는 oob score로 대체

## 결측치 열 제거후 학습진행

In [14]:
train_df = deepcopy(train_df_backup)
train_df.dropna(axis=1, inplace=True)
X,y = train_df.iloc[:,1:-1].to_numpy(), train_df.iloc[:,-1].to_numpy()
X.shape, y.shape

((1095, 7), (1095,))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=123456)
for i, state in enumerate(random_state_list): 
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(f'{i+1}번째')
    print(f'Score : {score}')
    print(f'Oob_Score : {rf.oob_score_}')

1번째
Score : 0.5788748632898335
Oob_Score : 0.5889070633734034
2번째
Score : 0.5692414347708896
Oob_Score : 0.5899570726638563
3번째
Score : 0.5657883832797002
Oob_Score : 0.5919857114415834
4번째
Score : 0.5748188575601721
Oob_Score : 0.5905783596662164
5번째
Score : 0.5762625386139495
Oob_Score : 0.5936226794748685
6번째
Score : 0.5725428318438057
Oob_Score : 0.5882763793050098
7번째
Score : 0.5718618949505052
Oob_Score : 0.5986665904702615
8번째
Score : 0.5655754841406344
Oob_Score : 0.5847002792930165
9번째
Score : 0.5692939997224675
Oob_Score : 0.5879207760226488
10번째
Score : 0.5741195911694021
Oob_Score : 0.5900711981363265


열 제거시 특징정보 부족으로 인한 성능저하

In [16]:
print('\033[31m'+'Feature importance'+'\033[0m')
print(*sorted(zip(train_df.iloc[:,1:-1].columns ,rf.feature_importances_), key=lambda x : x[1], reverse=True), sep='\n')

Feature importance
('temp_highest', 0.35640691036787103)
('temp_mean', 0.167271463869861)
('humidity', 0.15512559255709749)
('sunshine_rate', 0.10145969715370509)
('wind_mean', 0.09650547565027802)
('temp_lowest', 0.07638779685776384)
('wind_max', 0.04684306354342366)


# Null 값 조정

## 0으로 조정

In [17]:
train_df = deepcopy(train_df_backup)
train_df.fillna(0, inplace=True)

In [18]:
X,y = train_df.iloc[:,1:-1].to_numpy(), train_df.iloc[:,-1].to_numpy()
X.shape, y.shape

((1095, 11), (1095,))

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=123456)
for i, state in enumerate(random_state_list): 
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(f'{i+1}번째')
    print(f'Score : {score}')
    print(f'Oob_Score : {rf.oob_score_}')

1번째
Score : 0.7012665826413871
Oob_Score : 0.7257346536945285
2번째
Score : 0.7044820426987999
Oob_Score : 0.7170893690727659
3번째
Score : 0.710298096543602
Oob_Score : 0.7206832653786739
4번째
Score : 0.713289857423456
Oob_Score : 0.717905363256758
5번째
Score : 0.7120642075615111
Oob_Score : 0.7249620737302169
6번째
Score : 0.7141774268602189
Oob_Score : 0.7198691394070217
7번째
Score : 0.7105683473416868
Oob_Score : 0.7185902561016277
8번째
Score : 0.7043024834826039
Oob_Score : 0.7225667538865868
9번째
Score : 0.7077973707295924
Oob_Score : 0.7224219583863145
10번째
Score : 0.7069555690512074
Oob_Score : 0.7236276392709369


성능이 훨씬 개선됨

## 평균으로 조정

In [20]:
train_df = deepcopy(train_df_backup)
train_df.fillna(train_df.mean(), inplace=True)

/tmp/ipykernel_846/4105978821.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_df.fillna(train_df.mean(), inplace=True)


In [21]:
X,y = train_df.iloc[:,1:-1].to_numpy(), train_df.iloc[:,-1].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=123456)
for i, state in enumerate(random_state_list): 
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(f'{i+1}번째')
    print(f'Score : {score}')
    print(f'Oob_Score : {rf.oob_score_}')

1번째
Score : 0.652592922072871
Oob_Score : 0.6769183265556656
2번째
Score : 0.6605055546733306
Oob_Score : 0.676590314872101
3번째
Score : 0.6549938468263123
Oob_Score : 0.6733812207991839
4번째
Score : 0.6544877914972129
Oob_Score : 0.6782058191848656
5번째
Score : 0.6558501515771942
Oob_Score : 0.681414873410253
6번째
Score : 0.6513389822513183
Oob_Score : 0.6781367927695982
7번째
Score : 0.6606089255109857
Oob_Score : 0.6789747414470613
8번째
Score : 0.6568190306994113
Oob_Score : 0.6856706685752083
9번째
Score : 0.6572245720752512
Oob_Score : 0.6768391127026268
10번째
Score : 0.658081012368641
Oob_Score : 0.6748594710646043


## 중간값으로 조정

In [22]:
train_df = deepcopy(train_df_backup)
train_df.fillna(train_df.median(), inplace=True)

/tmp/ipykernel_846/1783643082.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_df.fillna(train_df.median(), inplace=True)


In [23]:
X,y = train_df.iloc[:,1:-1].to_numpy(), train_df.iloc[:,-1].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state=123456)
for i, state in enumerate(random_state_list): 
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(f'{i+1}번째')
    print(f'Score : {score}')
    print(f'Oob_Score : {rf.oob_score_}')

1번째
Score : 0.6806910898003065
Oob_Score : 0.682289484592056
2번째
Score : 0.6804293019255908
Oob_Score : 0.6724480589537025
3번째
Score : 0.6862484208653927
Oob_Score : 0.6776414508300805
4번째
Score : 0.6814621219010213
Oob_Score : 0.6737230163361949
5번째
Score : 0.6779468031058178
Oob_Score : 0.6813946123095453
6번째
Score : 0.6792837444377166
Oob_Score : 0.6771931641545716
7번째
Score : 0.6897729197928637
Oob_Score : 0.6793574684815246
8번째
Score : 0.6731543568683227
Oob_Score : 0.6784961982127113
9번째
Score : 0.6830733178521298
Oob_Score : 0.6835965910831119
10번째
Score : 0.684809005476505
Oob_Score : 0.6807492853093607
